# VacationPy
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/output_data/Weatherpy.csv")
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chlorakas,34.8042,32.4083,55.40,67,20,8.05,CY,2021-02-03
1,1,jamestown,42.0970,-79.2353,28.00,80,90,12.66,US,2021-02-03
2,2,bredasdorp,-34.5322,20.0403,57.20,94,0,3.44,ZA,2021-02-03
3,3,cherskiy,68.7500,161.3000,1.67,84,0,2.35,RU,2021-02-03
4,4,chuy,-33.6971,-53.4616,71.13,93,86,4.18,UY,2021-02-03


### Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store Lat and Lng into Locations
locations = cities_df[["Lat","Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)

# Create a humidity heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,max_intensity=100,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [5]:
ideal_conditions_df = cities_df.loc[(cities_df["Max temp"] > 70) & (cities_df["Max temp"] < 80) & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0)]
ideal_conditions_df = ideal_conditions_df.dropna(how="any")
ideal_conditions_df.head()

,Unnamed: 0,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
87,87,busselton,-33.6500,115.3333,79.00,65,0,8.99,AU,2021-02-03
144,144,along,28.1667,94.7667,74.08,30,0,1.01,IN,2021-02-03
191,191,kisangani,0.5167,25.2000,71.08,77,0,2.06,CD,2021-02-03
200,200,puerto madero,14.7208,-92.4215,77.61,65,0,7.02,MX,2021-02-03
204,204,tura,25.5198,90.2201,76.42,35,0,2.42,IN,2021-02-03


### Hotel Map

In [6]:
# Set up new hotel dataframe
hotel_df = ideal_conditions_df.loc[:,["City","Country","Lat","Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
87,busselton,AU,-33.6500,115.3333,
144,along,IN,28.1667,94.7667,
191,kisangani,CD,0.5167,25.2000,
200,puerto madero,MX,14.7208,-92.4215,
204,tura,IN,25.5198,90.2201,
257,santa luzia,BR,-19.7697,-43.8514,
297,puerto escondido,MX,15.8500,-97.0667,
306,panara,IN,22.0667,78.5500,
348,comodoro rivadavia,AR,-45.8667,-67.5000,
349,pisco,PE,-13.7000,-76.2167,


In [7]:
# set up parameters dictionary
params = {"type": "hotel", "keyword": "hotel", "radius": 5000, "key": g_key}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up loop through ideal_conditions data frame
for index, row in hotel_df.iterrows():
    
    # retrieve city, lat and lng from df
    city = row["City"]
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add keyword to parameters dictionary
    params["location"] = f"{lat},{lng}"
    
    # create url and call API request
    response = requests.get(base_url,params=params).json()
    
    results = response["results"]
    
    # Save the hotel name to dataframe
    try:
        print(f"There is a hotel in {city} named {results[0]['name']}")
        hotel_df.loc[index,"Hotel Name"] = results[0]["name"]
        
    except(KeyError,IndexError):
        print("missing field, skip")
        
print("----End of Search----")

There is a hotel in busselton named The Sebel Busselton
There is a hotel in along named Hotel West
There is a hotel in kisangani named Hotel Kisangani
There is a hotel in puerto madero named Mision Surf Mexico
There is a hotel in tura named Hotel Polo Orchid
There is a hotel in santa luzia named Hotel Casa Nova
There is a hotel in puerto escondido named Selina Puerto Escondido
There is a hotel in panara named pillu hotel katkuhi
There is a hotel in comodoro rivadavia named Lucania Palazzo Hotel
There is a hotel in pisco named HOSTAL MIRAMAR
There is a hotel in iztapa named Hotel EcoCabañas Resort
There is a hotel in yenagoa named De Brass Suites
There is a hotel in pochutla named Hotel Posada San Jose
missing field, skip
There is a hotel in boa vista named Hotel Ajuricaba
There is a hotel in keffi named Minki Suites LTD
There is a hotel in kassala named Hipton Hotel
missing field, skip
There is a hotel in sao filipe named Hotel Xaguate
There is a hotel in wamba named Roma Zee Guest Hou

In [8]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
87,busselton,AU,-33.6500,115.3333,The Sebel Busselton
144,along,IN,28.1667,94.7667,Hotel West
191,kisangani,CD,0.5167,25.2000,Hotel Kisangani
200,puerto madero,MX,14.7208,-92.4215,Mision Surf Mexico
204,tura,IN,25.5198,90.2201,Hotel Polo Orchid
257,santa luzia,BR,-19.7697,-43.8514,Hotel Casa Nova
297,puerto escondido,MX,15.8500,-97.0667,Selina Puerto Escondido
306,panara,IN,22.0667,78.5500,pillu hotel katkuhi
348,comodoro rivadavia,AR,-45.8667,-67.5000,Lucania Palazzo Hotel
349,pisco,PE,-13.7000,-76.2167,HOSTAL MIRAMAR


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))